In [1]:
from imitation_learning.neural_network import RemovalTimePredictor
import os
import torch
import re
from imitation_learning.train import load_graph_data

def load_model(folder, model_file_name=None, device='cpu'):
    full_folder  ='work_dir/'+folder
    if model_file_name is None:
        # Get latest file
        dirFiles = os.listdir(full_folder)

        def key(f):
            return int(re.sub('\D', '', f.split('_')[1]))
        print(dirFiles)
        # get number from file name CNN_04-21-2021-14-46-05_1000 -> 04212021144605 and sort on that number
        dirFiles.sort(key=key)
        model_file_name = dirFiles[-1]
    print("Loading model: ", model_file_name)
    path = full_folder + "/" + model_file_name
    model = RemovalTimePredictor()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model
base_path = 'chsp-generators-main/instances/'
files = [base_path+f"linear_solutions_{i}.npy" for i in range(1,2)] # Can choose other solution files

model = load_model('exp-s1-idrandom_split_big3', model_file_name='model_315.pt') # Model trained on many data.
model = load_model('exp-s1-idrandom_split', model_file_name='model_915.pt') # Model trained on less data but longer and performs better


Loading model:  model_315.pt
Loading model:  model_915.pt


In [2]:
from math import sqrt
import numpy as np
from torch import nn, optim
criterion = nn.MSELoss()
train_set, test_set, training_input_data = load_graph_data(files)


# Select data here
data = train_set[0]
graph_inputs, labels = data

output, edges = model(graph_inputs, graph_inputs.ndata['x'], graph_inputs.edata['w'])
prediction = edges[1:graph_inputs.num_nodes(), 0, 0]
print("prediction", prediction), print("labels", labels[0])
print("prediction", len(prediction)), print("labels", len(labels[0]))

"RMSE: " + str(sqrt(criterion(prediction, labels[0])))
graph_inputs

loading chsp-generators-main/instances/linear_solutions_1.npy
Shape input (2994, 2)
prediction tensor([229.8911, 299.7462, 350.8931, 673.4486, 429.4846, 647.4657, 875.1756],
       grad_fn=<SelectBackward0>)
labels tensor([224., 343., 395., 700., 433., 612., 845.])
prediction 7
labels 7


Graph(num_nodes=8, num_edges=64,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'w': Scheme(shape=(1,), dtype=torch.float32)})

In [8]:
"Problem input:", training_input_data[0]

('Problem input:',
 {'Ninner': 6,
  'tmin': [470, 790, 539, 211, 413, 680],
  'tmax': [1132, 7284, 3853, 1685, 613, 3059],
  'f': [224, 119, 52, 31, 100, 179, 50],
  'e': array([[16,  0,  9, 13, 22, 29, 42],
         [25,  9,  0,  4, 13, 20, 33],
         [29, 13,  4,  0,  9, 16, 29],
         [38, 22, 13,  9,  0,  7, 20],
         [45, 29, 20, 16,  7,  0, 13],
         [58, 42, 33, 29, 20, 13,  0],
         [ 0, 16, 25, 29, 38, 45, 58]]),
  'emptys': [16, 9, 4, 9, 7, 13, 13]})